In [1]:
from pyibex import * 

In [25]:
from pyibex import *
from tubex_lib import *
tdomain = Interval(0,10)
x1 = Tube(tdomain)
dir(x1)
# 100-slices tubes:
dt = 0.1
x3 = Tube(tdomain, dt, Interval(0,2))            # [0,10]→[0,2]

traj = TrajectoryVector(tdomain, TFunction("(sin(t) ; cos(t) ; cos(t)+t/10)"))

x8 = Tube(traj[0], dt)           # 100 slices tube enclosing sin(t)

traj = TrajectoryVector(tdomain, TFunction("exp(t)"))
x8 = Tube(traj[0], dt)    
x8.index()

TypeError: index(): incompatible function arguments. The following argument types are supported:
    1. (self: tubex_lib.tube.Tube, slice: tubex::Slice) -> int

Invoked with: Tube [0, 10]↦([0.9999999999999998, 22026.46579480673]), 100 slices

In [41]:
#dir(x8.slice())
dir(Interval(0,1))
Interval(0,1).lb()
x = Interval(0,1)
dir(x)
2 in x
x.bisect()

([0, 0.5], [0.5, 1])

In [73]:
Interval(Interval(-oo,oo).lb(),0)

[-inf, 0]

So what's my problem:
The tube interface isn't pythonic and doiesn't give me enoughb access.

And rolling my own is...?
Well I don't get the contractors by default.

If I'm just getting Intervals, maybe i should work in julia where people might care
But the bidirectional yield beavior of python was going to be cute.


struct BiChannel{A,B}
    
end

function refiner(c1)
    recv = Channel()
    send = Channel() do send
        put!(send, constraints)
        model = take!(recv)
    end
    (send, recv)
end

put!(c{A,B}, x::A) = 
put!(c{A,B})

It's not cute anymore



f(x0,t) # an implicit function of the initial condition and time

if linear we can seperate out initial condition dependence:
x0 f_1(t) + f_2(t)



A bounded model check of a hybrid system should be paramterized by number of mode shifts,
not by "dt"?


Could I use the error mechabism?
Throw a continuation value.

throw (c -> 

)

Not super sure this would work


Ok we don't have to go for such a sophisticated refinement method.
We could just check if in tube. If not, recompute at dt/2


In [86]:
dir(Interval(-oo,oo))
Interval(0,1).is_unbounded()

False

In [ ]:

def refine_int(i,t):
    if i.ub() == Inf and i.lb() == -Inf:
        if i.ub() == Inf and i.lb() == -Inf:
        
    = Interval(i.lb, t), Interval(t, i.ub)
    

In [2]:
class MyTube():
    def __init__(self, f):
        self.ts = [Interval(-oo,oo)] #[Interval(-oo,-10), Interval(-10,10) , Interval(10,oo)] #[Interval(-oo,oo)]#
        self.f = f
        self.ys = [f(t) for t in self.ts]
    def refine(self,t):
        fresh = []
        for i in range(len(self.ts)):
            if t in self.ts[i]:

                # rather than bisect use refinement exactly at.
                # hmm. what about a repeated refinement?
                # I guess we should bisect and sample
                # or we should only sample when semi infinite refinement
                tint = self.ts[i]
                #print(t, tint, t in tint)
                if tint.is_unbounded():
                    lt,ut =  Interval(tint.lb(), t), Interval(t, tint.ub())
                else:
                    lt,ut = tint.bisect()
                
                self.ts[i] = lt
                y = self.f(lt)
                self.ys[i] = y
                fresh.append((lt,y))
                
                self.ts.append(ut)
                y = self.f(ut)
                self.ys.append(y)
                fresh.append((ut,y))
        return fresh
    def __call__(self,t):
        for tt,y in zip(self.ts,self.ys):
            if t in tt:
                return y 
    def __repr__(self):
        return repr(list(zip(self.ts, self.ys )))


In [170]:
texp = MyTube(exp)
texp.refine(0)
texp.refine(1)
texp.refine(0.75)
#texp(1)
texp





0 [-10, 10] True
1 [0, 10] True
0.75 [0, 5] True


[([-inf, -10], [0, 4.539992976248487e-05]), ([-10, 0], [4.539992976248484e-05, 1.000000000000001]), ([10, inf], [22026.46579480671, inf]), ([0, 2.5], [0.9999999999999998, 12.18249396070348]), ([5, 10], [148.4131591025765, 22026.46579480673]), ([2.5, 5], [12.18249396070347, 148.4131591025767])]

In [171]:
Interval(-oo,oo).lb() == -oo


True

In [3]:
def z3_in(x, i):
    # check for infinities
    c = []
    if i.ub() != oo:
        c += [x <= i.ub()]
    if i.lb() != -oo:
        c += [ i.lb() <= x]
    return And(c)

def z3_exp(x):
    fexp = Function("exp", RealSort(), RealSort())
    def res():
        tube = MyTube(exp)
        m = yield [ fexp(x) >= 1 + x, fexp(x) >= 0 ]
        while True:
            #print(fexp(x))
            #print(dir(m.eval(fexp(x))))
            #if m.eval(fexp(x)) in tube(m.eval(x)):
            #    c = []
            #else:
            x0 = m.eval(x).numerator_as_long() / m.eval(x).denominator_as_long() 
            #print(x0)
            f = tube.refine(x0)
            print(tube)
            #print(f)
            c = [ Implies( z3_in(x,t) ,  z3_in(fexp(x), y)) for t, y in f]
            m = yield c
    return fexp(x), res

def z3_tube(name, x, f):
    f_z3 = Function(name, Realsort(), Realsort())
    def res():
        tube = MyTube(f)
        m = yield [ Implies( z3_in(x,t) ,  z3_in(fexp(x), y)) for t, y in tube]
        while True:
            if m[f_z3(x)] in tube(m[x]):
                c = []
            else:
                f = tube.refine(m[x])
                c = [ Implies( z3_in(x,t) ,  z3_in(fexp(x), y)) for t, y in f]
            m = yield c
            
    return f_z3(x), res
    

In [15]:
from z3 import *
x = Real("x")

y, c = z3_exp(x)

c = c()
s = Solver()
s.add(next(c))
s.add(Not( Implies( x >= 1, y >= 2.7 )))
s.check()
m = s.model()
for i in range(7):
     if s.check() == unsat:
            print("proved")
            print(i)
            break
     m = s.model()
     newcon = c.send(m) 
     s.add( newcon )



[([-inf, 1], [0, 2.718281828459046]), ([1, inf], [2.718281828459044, inf])]
proved
1


In [10]:
from z3 import *
x = Real("x")

y, c = z3_exp(x)

c = c()
s = Solver()
s.add(next(c))
s.add( x >= y )
print(s.check())
m = s.model()
print(m.eval(x).as_decimal(10))
for i in range(11):
     if s.check() == unsat:
            print("proved")
            break
     m = s.model()
     print(m[x].as_decimal(10))
     #print(dir(m))
     newcon = c.send(m) 
     print(newcon)
     #print(newcon)
     s.add( newcon )

unsat


Z3Exception: model is not available

In [112]:
import numpy as np
np.exp(1)

2.7182818284590455

In [ ]:
class MyTube():
    def __init__(self):
        self.ts = np.linspace(0,1,10)
        self.intts = [ Interval(i,j) for i,j in  zip(self.ts[:-1], self.ts[1:])]
        self.ints = [ exp( ) for i in  self.intts[] ]
    
    def refine(self,t):
        for ind in range( len(self.ints) ):
            i = self.intts[ind]
            fresh = []
            if t in i:
                x,y = i.bisect()
                self.intts[ind] = x
                self.ints[ind] = exp(x)
                fresh.append( (x,exp(x)))
                
                self.intts.append(y)
                self.ints.append(exp(y))
                fresh.append( (y,exp(y))
        return fresh
        
    def 
        '''
                upper = len(self.intts) 
        lower = 0
        while True
            ind = (upper + lower) / 2
            i = self.intts[ind]
            if t in i:
                break
            elif t < ind:
                upper = ind
            elif t > ind:
                lower = ind
        '''
        
                
                
            





Move my sights from building the bounded differential equation bound to mewrely checking it.

Given a tube, what conditions 
x <= Lx is the condition I think on the tube x.
Ix = y injection into the domain of Sets of trajectories
Ay = x the hull into x

y <= IAy
AIx <= x

galois connection
Ay <= x  <->  y <= Ix


x <= Lx

L is monotonic, but it is not necessarily contractive.
L is derived from K.
L = AKI




in what sense does it containt






What is an interval relation?
A pair of functions to :: I^m -> I^n, from :: I^n -> I^m
With consistnency conditions. to . from <= id   and from . to <= id

forall x. to(from(x)) <= x   is what that means
contractive in both directions.
Well they should both be monotonic functions anyhow.

There is an ordering relation on intervals. Set inclusion

Intervals are a lattice.

In categorical terminology then, to and from are functors.



https://arxiv.org/pdf/1404.7171.pdf dreal paper descrbing their thing


I don't quite understand where people put their boot strap to the continuum



https://www.ensta-bretagne.fr/jaulin/paper_chabert_quimper.pdf constractor programming


In [239]:
Interval.EMPTY_SET
Interval.ALL_REALS
Interval(-oo,oo)
Interval()
Interval(-2,3)
IntervalVector(2,[1,3])
IntervalVector([1,2,3])

(<1, 1> ; <2, 2> ; <3, 3>)

In [241]:
dir(exp(Interval(0,1)))

['ALL_REALS',
 'EMPTY_SET',
 'HALF_PI',
 'NEG_REALS',
 'ONE',
 'PI',
 'POS_REALS',
 'TWO_PI',
 'ZERO',
 '__abs__',
 '__add__',
 '__and__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__isub__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pow__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 'assign',
 'bisect',
 'complementary',
 'contains',
 'copy',
 'diam',
 'diff',
 'inflate',
 'interior_contains',
 'intersects',
 'is_bisectable',
 'is_degenerated',
 'is_disjoint',
 'is_empty',
 'is_interior_subset',
 'is_strict_interior_s

The simplest version of something that might be called an exact real 
is to use already available interval functionality.

(Turn I -> I function into (epsx -> xI) -> (epsy -> yI) function



It's a goofy little machine that searches for the right input epsilon.
Very inefficient

Still, I think it basically makes sense.

We've lifted something monadic to full m x -> m y form?




In [ ]:
def lift1(f):
    def res1(x):
        def res(epsy):
            epsx = 1
            yI = Interval(0).widen(epsy+1)
            while width(yI) > epsy:
                epsx = epsx/2
                yI = f(x(epsx))
            return yI
        return res
    return res1

def lift2(f):
    def res1(x,y):
        def res(epsz):
            epsx = 1
            zI = Interval(0).widen(epsz+1)
            while width(zI) > epsz:
                epsx = epsx/2
                zI = f(x(epsx), y(epsx))
            return zI
        return res
    return res1

rsin = lift1(sin)



This implem,entation is nice because it's pure, but it's better to switch the generators.
We don't need to rerun the loops every time because we're only asking for refinements

now we represented 
eps -> interval
instead by 
generator that yileds intervals


We make things tighter on demand.



Exact reals an differentiation are very similar

Any constructibly continuous functin has a function associated with it that gives the modulus of continuity.
epsilon(x,epsy)

Any uniformly continuous function has a function that just needs to take in epsx(epsy).

You can obviously constructively compose these functions to get a valid continuity function for the composition for the uniform case.  It takes no cleverness

lambda epsz: epsx(epsy(epsz))

This is the analog of combining the derivatives of a composition of linear functions

However, if not uniformly continuous, we need dependence on the base point.

lambda compose(epsx, epsy):
       def res(x):
        epsx = epsx(x)
        epsy = epsy(f(x))
        return lambda epsz: epsx(epsy(epsz))
      reutrn res

In partilar the sort of backwards feeling definition of continuity has a similar flavor to the backwards running operation of reverse mode. They are both lenses.

This was my point in a previous post.

Actually, my point there was interesting. If you order epsilon first and go backwards forwards rather than forwards backwards.
Hmmm.




So the values are no longer particular intervals, but the _ability_ to request intervals computed at arbitrary accuracy.
This is a strange, subtle, almost philosophical difference.


what about just
interval -> interval functions? 
Yes, because of the spooky global dps parameter, these are valid as real -> real functions
 
Wait, does the dps parameter destory things that already existed? 
No that's fine in my global version. But i wonder about the 


A cut takes value in sierpinski space because it's a half line. It includes infinty.
The cut is not a point.
Ah but in a sense, an interval as the conjunction of two cuts.
Or that is a way to think about it.


We could make cuts using half infinite iv
Or using regular floating point in a rounding mode



Try using arb
Try to understand



In [56]:
iv.dps = 100
x = iv.mpf("0.1")
print(x)
iv.dps = 2
print(x)
y = iv.sin(x)
print(y)
iv.dps = 100
print(x)
print(y)
iv.mpf(["0","inf"])
iv.mpf(["-inf","0"])
# delta mid a b


[0.0999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999992856, 0.100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000179]
[0.1, 0.1]
[0.09973145, 0.09985352]
[0.0999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999992856, 0.100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000179]
[0.0997314453125, 0.099853515625]


mpi('+inf', '+inf')

In [58]:
from dataclasses import dataclass

semiinterval = interval

@dataclass(eq = False)
class Cut:
    l : Callable[[],semiinterval]
    r : Callable[[],semiinterval]


iv.dps = 15
iv.mpf("0.1")
iv.mpf(["0.1", "0.2"])
x = iv.mpf("1")
iv.sin(x)





In [ ]:

def upper_cut(x): # UPPER CUT WO PAAAAAHHH!!!
    return lambda : iv.mpf(x(), "inf")
def lower_cut(x):
    return lambda : iv.mpf("-inf", x())

def dirichlet(u,l):
    return lambda : iv.intersect(u(), l())

# the marshallo cut construct is a quantifer with two bodies. Odd.

def cut(upper,lower):
    # some kind of search for best currently available upper and lower bounds
    xu =
    xl =
    upper(xu) 

def cut(f : Callable[ real ], Tuple[sierp,sierp]): #return a tuple of refinable values. The fan of the above
    

In [76]:
from mpmath import iv
from typing import List, Set, Dict, Tuple, Optional
from typing import Callable, Iterator, Union, Optional, List
import operator as op

# type definitions
interval = mpmath.ctx_iv.ivmpf
# so we could just wrap a loop around thunked numbers
real = Callable[[], interval]

# Thunk[x] = Callable[[],x]
# delay[x], defer[x]

# lifts a one argument interval function to a one argument thunked interval function
def lift(f : Callable[[interval], interval]) -> Callable[[real], real]:
    def res(x : real) -> real:
        return lambda : f(x())  
    return res

# lifts a one argument interval function to a one argument thunked interval function
def lift2(f : Callable[[interval, interval], interval]) -> Callable[[real, real], real]:
    def res(x : real, y: real) -> real:
        return lambda : f(x(), y())  
    return res

def const(x : str) -> real:
    return lambda : iv.mpf(x)

# some simple lifts of mpmath functions
rsin = lift(iv.sin)
rcos = lift(iv.cos)
rexp = lift(iv.exp)
rlog = lift(iv.log)
rneg = lift(op.neg)
radd = lift2(op.add)
rmul = lift2(op.mul)


third = const("1/3")
# a random example calculation
v = rmul( rexp( third) , rlog(third))

for i in range(10):
    iv.dps = i
    print(v())

'''
class Real():
    def __init__(self, f):
        self.f = f
    def __repr__(self):
    sin = lift(iv.sin)
    cos = lift(iv.cos)
    exp = lift.exp
'''



belnap = Callable[[], Optional[bool]] 

# lifts a one argument interval function to a one argument thunked interval function
def lift2(f : Callable[[interval, interval], Optional[bool]]) -> Callable[[real, real], belnap]:
    def res(x : real, y: real) -> belnap:
        return lambda : f(x(), y())  
    return res


rlt = lift2(op.lt)
rgt = lift2(op.gt)
rne = lift2(op.ne)

# no actually. We actually need truth tables.
lift(op.not_)
lift(op.and_)
lift(op.or_)

# total poison. Unacceptable?   
req = lift2(op.eq)
rge = lift2(op.ge)
rle = lift2(op.le)

third = const("1/3")
thirdish = const("0.3333")
v = rgt(third, thirdish)
for i in range(10):
    iv.dps = i
    print(f"Precision {i}: {v()}")
    #print(third())
    #print(thirdish())

[-2.0, -1.0]
[-1.5625, -1.48438]
[-1.537109, -1.529297]
[-1.5339355, -1.5327148]
[-1.53326416, -1.53320313]
[-1.533239365, -1.533231735]
[-1.5332376957, -1.5332362652]
[-1.53323698044, -1.53323693573]
[-1.533236963674, -1.533236958086]
[-1.5332369611133, -1.5332369599491]
Precision 0: None
Precision 1: None
Precision 2: None
Precision 3: None
Precision 4: True
Precision 5: True
Precision 6: True
Precision 7: True
Precision 8: True
Precision 9: True


In [90]:
# Though an accident of the short circuiting mechanism and falsity of None

x = [True, False, None]
for i in x:
    for j in x:
        print( f"{i} or {j} = {i or j}" )
for i in x:
    for j in x:
        print( f"{i} and {j} = {i and j}" )
for i in x:
        print( f"not {i} = {not i}" )
# nope. None and False = False. It was close though.

True or True = True
True or False = True
True or None = True
False or True = True
False or False = False
False or None = None
None or True = True
None or False = False
None or None = None
True and True = True
True and False = False
True and None = None
False and True = False
False and False = False
False and None = False
None and True = None
None and False = None
None and None = None
not True = False
not False = True
not None = True


In [91]:
belnap = Optional[bool]
def belor(x : belnap,y : belnap) -> belnap:
    if x == None and y == False:
        return None
    return x or y

def beland(x : belnap,y : belnap) -> belnap:
    if x == None and y == False:
        return False
    return x and y    
def belnot(x : belnap) -> belnap:
    if x == None:
        return None
    return not x


In [179]:
# As generators. The advantage is that we can retain information between refinements. Bad is that everything is just more complicated
# and the requirements of the calling pattern is stricter


def const(x):
    old_dps
    old_iv
    while True:
        if iv.dps < old_dps: # we can cache previous runs if iv increases or what have you.
            old_dps = iv.dps
            old_id = iv.mpf(x)
            yield old_iv:
        else:
            yield old_iv
    
def cos
        

        
v = const("0")
for i in range(5):
    iv.dps = i
    print(f"Precision {i}: {next(v)}")






Precision 0: [0.0, 0.0]
Precision 1: [0.0, 0.0]
Precision 2: [0.0, 0.0]
Precision 3: [0.0, 0.0]
Precision 4: [0.0, 0.0]


With an interval matrix solver, could we not make a precise formulation of the tube operator?




In [ ]:
# Reifying functions into tubes
# It's the analog of tabulate = [(x, f(x) for x in range(7)]
# tabulating 
# can be useful for memoization for example. We could use some kind of spatial data structure for fast indexing.
# Well, really we know roughly where to look in the tube for anyquery.


def tube(f : Callable[[real], real ]) -> Tube[[real],real]:
    return lambda : [(i(), f(i)()) for i in partition()]


a constraction mapping

def fix( f : (R -> R) -> (R -> R)  ) -> (R -> R)

Partition is what makes it searchabvle. Related to compactnessness. A finite cover

In [ ]:

# fix: find a fixed point of a function
def fix(f : Callable[[real], real]) -> real:
def min(f):
    min([f(i) for i in partition()])
def max(f):
def argmin(f): # stronger. Like epsilon operatros
    bestf = iv.mpf("inf")
    besti = None
    for i in partition()
        fi = f(i)
        if fi <= bestf == True:
            besti = i
            bestf = fi
        elif f(i) <= bestf == None:
            besti = i.union(besti)
            bestf = min(bestf, fi)
def argmax(f):

    
    
    

In [235]:

iv.dps = 3

x0 = iv.mpf("1")

tube = [x0] + [ iv.mpf([-10,10]) for _ in range(10)]

dt = iv.mpf([0,"1/10"])

def f(t,x): # xdot = x is expoentnial
    return x

print(dt)
from scipy.integrate import solve_ivp

sol = solve_ivp(f, [0,1], [1], t_eval=np.linspace(0.05,1.05,10,endpoint=False))
print(sol.t)
print(sol.y)

def f(t,x): # xdot = x is expoentnial
    return x - sol.y[0][t]

tube = [iv.mpf("0")] + [ iv.mpf([-10,10]) for _ in range(10)]
for i in range(10):
    for j in range(10):
        newtube = tube[i] + dt * f(i,tube[i+1])
        #print(newtube in tube[i+1])
        #newtube = iv.mpf(  [ min( tube[i], newtube ) ,  max( tube[i], newtube )])  
        tube[i+1] = newtube
print([ e + y for e,y in zip(tube[1:], sol.y[0])])
print(np.exp(0.1))
import numpy as np
import matplotlib.pyplot as plt

#plt.plot([float(mpmath.nstr(p.a)) for p in tube])
#print(dir(x0))

#iv.mpf([x0.a,x0.b + 1, x0.b])
#dir(iv)

[0.0, 0.1000061]
[0.05 0.15 0.25 0.35 0.45 0.55 0.65 0.75 0.85 0.95]
[[1.0512711  1.16184244 1.28406684 1.41911975 1.56833666 1.73322614
  1.91546986 2.11692255 2.33961202 2.58573918]]
[mpi('0.934326', '1.05151'), mpi('0.902588', '1.16187'), mpi('0.853394', '1.28418'), mpi('0.782715', '1.41919'), mpi('0.68689', '1.56836'), mpi('0.561279', '1.7334'), mpi('0.400146', '1.91553'), mpi('0.197754', '2.11719'), mpi('-0.0532227', '2.33984'), mpi('-0.36084', '2.58594')]
1.1051709180756477


In [ ]:
def ode(f, x0, t0):
    # step one, send off f to ode int
    # Then use interval methods on the error equation
    # any particular step size is not necessarily going to be sufficet to have a contraction mapping
    # it's only monotone on the subset of the lattice that contains the fixed point.
    # also we have to search outward to make sure we enclose the solution.
    # consider a finite difference system where we use intervals. It's an interesting analog.
    
    # 1 find contractive regime
    bounds = 10
    dps = iv.dps
    while True:
        tube = tubes(-bounds,bounds, dps)
        with iv.dps = dps:
            x = x0() # hmmm. How do we do initial conditions. We can't know exactly what position this came from.
            dt = iv.mpf(f"1/{10**iv.dps}")
            for i in partition():
                x = x + f( tube(i) ) * dt
                
                
        bounds = bounds * 10
        dps += 1

In [88]:
x = [True, False, None]
for i in x:
    for j in x:
        print( f"{i} or {j} = {belor(i, j)}" )
for i in x:
    for j in x:
        print( f"{i} and {j} = {beland(i,j)}" )

True or True = True
True or False = True
True or None = True
False or True = True
False or False = False
False or None = None
None or True = True
None or False = None
None or None = None
True and True = True
True and False = False
True and None = None
False and True = False
False and False = False
False and None = False
None and True = None
None and False = False
None and None = None


In [124]:
def belnot(x):
    if x == None:
        return None
    return not x
def beland(x,y):
    if y == None:
        x,y = y,x
    if x == None:
        if y == True:
            return None
        elif y == False:
            return False
        else:
            assert(y == None)
            return None
    return x and y

def belor(x,y):
    if x == None and y != True:
        return None
    if y == None and x != True:
        return None
    return x and y

rbelnap = Callable[[], belnap]

# no actually. We actually need truth tables.
rnot = lift(belnot)
ror = lift2(belor)
rand = lift(beland)


In [160]:
predicate = Callable[[real], rbelnap]
#def thunk()
#predicate = Callable[[interval], belnap] # The previous one is the refinement lifting of this.
#predicate = Callable[[],Callable[[interval], belnap]]

def partition(): #paving?
    d = 10**iv.dps # denominator = number of partitions
    for i in range(d):
        yield iv.mpf( [f"{i}/{d}" , f"{i+1}/{d}" ])

        
def exists(f):
    def res():
        allfalse = True
        for i in partition():
            test = f(i)
            if test == True:
                return True
            elif test == None:
                allfalse = False
        if allfalse:
            return False
        return None
    return res

def forall(f : predicate) -> rbelnap:
    pass
    
# find could also grow out the refinement region to infinity
def find( f : predicate ) -> real:
    def res():
        for i in partition():
            test = f(lambda : i)()
            if test == True:
                return i
        return iv.mpf(["0","1"])
    return res

def exists(f):
    return f(find(f))
def forall(f):
    return rnot( find( lambda x: rnot(f(x)) ) )
# here brute forcing at every refinement is obviously shitty.
# We should exclude regions as early as possible. To do so we'd need to maintain state.

v = find( lambda x: rgt( rmul(x,x),  const("1/4")) )
#v = forall( lambda x: rgt( rmul(x,x),  const("1/4")) )
#v = exists( lambda x: rgt( rmul(x,x),  const("1/4")) )
for i in range(5):
    iv.dps = i
    print(f"Precision {i}: {v()}")

Precision 0: [0.625, 0.875]
Precision 1: [0.59375, 0.703125]
Precision 2: [0.5097656, 0.5205078]
Precision 3: [0.50097656, 0.5020752]
Precision 4: [0.500099182, 0.500205994]


In [177]:
def integral(f : Callable[[real], real]):
    def res():
        return sum([ f(lambda : i)() for i in partition() ]) * const(f"1/{10**iv.dps}")()
    return res

v = integral( lambda x : rmul(x,x) )
for i in range(3):
    iv.dps = i
    print(f"Precision {i}: {v()}")

Precision 0: [0.1875, 0.625]
Precision 1: [0.273438, 0.398438]
Precision 2: [0.3222656, 0.3447266]


In [150]:
dir(iv.mpf(["0","1"]))
iv.mpf(["0", "1"]).delta

mpi('1.0', '1.0')

In [ ]:
path = Callable[[real], real]

# R ^2


So I've heard the stories about equality being a no-no and whatforth, but I'm just not really seeing some of it. Is that because I'm not using primitive enough constructions? Half intervals?




Sierpinski space is not showing up.






In [96]:
iv.dps = 2
def partition():
    d = 10**iv.dps # denominator = number of partitions
    for i in range(d):
        yield iv.mpf( [f"{i}/{d}" , f"{i+1}/{d}" ])
        
for i in partition():
    print(i)

[0.0, 0.01000977]
[0.009994507, 0.02001953]
[0.01998901, 0.0300293]
[0.02999878, 0.04003906]
[0.03997803, 0.05004883]
[0.04998779, 0.06005859]
[0.05999756, 0.07006836]
[0.06994629, 0.08007813]
[0.07995605, 0.09008789]
[0.08996582, 0.1000977]
[0.09997559, 0.1101074]
[0.1099854, 0.1201172]
[0.1199951, 0.130127]
[0.1298828, 0.1401367]
[0.1398926, 0.1501465]
[0.1499023, 0.1601563]
[0.1599121, 0.170166]
[0.1699219, 0.1801758]
[0.1799316, 0.1901855]
[0.1899414, 0.2001953]
[0.1999512, 0.2102051]
[0.2099609, 0.2202148]
[0.2199707, 0.2302246]
[0.2299805, 0.2402344]
[0.2399902, 0.25]
[0.25, 0.2602539]
[0.2597656, 0.2700195]
[0.2695313, 0.2802734]
[0.2797852, 0.2900391]
[0.2895508, 0.300293]
[0.2998047, 0.3100586]
[0.3095703, 0.3203125]
[0.3198242, 0.3300781]
[0.3295898, 0.340332]
[0.3398438, 0.3500977]
[0.3496094, 0.3603516]
[0.3598633, 0.3701172]
[0.3696289, 0.3803711]
[0.3798828, 0.3901367]
[0.3896484, 0.4003906]
[0.3999023, 0.4101563]
[0.409668, 0.4204102]
[0.4199219, 0.4301758]
[0.4296875, 0

In [27]:
import mpmath
type(iv.mpf("0.2"))
Callable[[],mpmath.ctx_iv.ivmpf]

typing.Callable[[], mpmath.ctx_iv.ivmpf]

In [51]:
!pip3 install python-flint

  Using cached python-flint-0.3.0.tar.gz (109 kB)
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-o_phpml9/python-flint/setup.py'"'"'; __file__='"'"'/tmp/pip-install-o_phpml9/python-flint/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-evejo1zy
       cwd: /tmp/pip-install-o_phpml9/python-flint/
  Complete output (23 lines):
  running bdist_wheel
  running build
  running build_ext
  cythoning src/pyflint.pyx to src/pyflint.c
  /home/philip/.local/lib/python3.8/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/pip-install-o_phpml9/python-flint/src/flint.pxd
    tree = Parsing.p_module(s, pxd, full_module_na

In [ ]:

def lift(f):
    def res(x):
        prec = 1
        for approxx in x:
            while approxx.width() < prec: #reuse approxx
                prec = prec/2
                set_precision(prec)
                approxy = f(approxx)
                yield approxy
    return res

def const(x):
    while True:
        yield Interval(x)


In [ ]:
# Less than is a predicate that returns a true value
# this is a refinable boolean
def lt(x,y):
    for x,y in zip(x,y):
        if x < y:
            while True:
                yield True
        elif y < x: # is this wrong in some sense?
            while True:
                yield False
        yield None

In [11]:
f = Function("x","y","x * sin(y)")
f.eval(IntervalVector([[1,2],[3,4]]))

[-1.513604990615857, 0.2822400161197349]

In [12]:
ctc = CtcFwdBwd(f, [0,oo])

In [ ]:


#interval constraint programming

constraints = []

store = {} # index from variable to value of variable
index = # index from variables to constraints that use them



for c in constraints:
    inputs = map( x -> store[x] , c.inputs)
    res = c.func(inputs...)
    outval = store[c.output]
    if out & res != outval:
        store[out] = outval & res
        # active.append( index[c.output] )
    
check_list = constraints

store = {   }

while check_list != []
    c = check_list.pop()
    update_store(c,store)
    

    




In [3]:
from pyibex import *
from tubex_lib import *

x = Tube(Interval(0,1), 0.01, TFunction("cos(t) + abs(t-5)*[-0.1,0.1]"))

beginDrawing()
fig = VIBesFigTube("My first tube")
fig.add_tube(x, "x")
fig.show()
endDrawing()

In [9]:
tdomain = Interval(0,10)
x = Trajectory(tdomain, TFunction("cos(t) + sin(2*t)"))
x(1)
x.codomain()
x.tdomain()
x.diff()

Trajectory [0, 10]↦[-3, 3] (Fnc object)

Contractor networks. 

This feels like it fits in the general scheme of propagators.
DP sanders had a contractor thing right?
https://github.com/JuliaIntervals/IntervalConstraintProgramming.jl

Applied Interval Analysis, Luc Jaulin, Michel Kieffer, Olivier Didrit, Eric Walter (2001)
Introduction to the Algebra of Separators with Application to Path Planning, Luc Jaulin and Benoît Desrochers, Engineering Applications of Artificial Intelligence 33, 141–147 (2014)

https://www.ensta-bretagne.fr/jaulin/iamooc.html


In [4]:

f = Function("x", "y", "x * cos(x - y) + y")
sep = SepFwdBwd(f, CmpOp.LEQ)
X0 = IntervalVector(2, [-10,10])

X0


([-10, 10] ; [-10, 10])

In [1]:
from pyibex import *
from tubex_lib import *


In [5]:
x = Interval()
x = Interval(0,10)

x = IntervalVector(2, [-1,2])


ctc_add = CtcFunction(Function("x", "y", "a", "x+y-a "))



# how would I implement this?
x + y - a => calculate. #would give some interval.

Could store both forward and inverse function.

(x + y) - a == 0

(x0 - a)





interval closure. Sure.


galois connection between Lattice sets of R and their interval closures. Lattice of intervals

R^n boxes.functions of sets vs functions of points. You can derive a function of sets from a function fo points.

Monotonically increasing. Functions that make sense (are derived from point functions) are those that are monotonically increasing and convergent. When the set is a point, it equals the point function and monotnically increasing
wdith of input converging implies width of output converging



x sub y
f(x) sub f(y)


injection and closure. Every interval is a set.
Every set has a closure.

Convex hull is similar. BVetween the lattice of sets of R^n

<= 

What is a set?



set inversion. compute preimage


sivia - set inversion by interval analysis




if s

contractors

contractor is IR^n -> IR^n
C([x])
1. C([x]) sub [x] contractance
2. C([x] intersect X = [x] interesect x - completeness


A sub R^n. 
C(C(x)) sub C([x])

idempotent


so a self map of a lattice.
It's an endofunctor from category theory


Is it a homomorphism? 


motonoicity
[x] sub [y] implies C


C x -> x   looks like return or coreturn 
C C x -> C x   yes.

A contractor is a monad (or comonad)




True and False as a lattice. The Maybe lattice.
Julia's missing


contract(x)


primitive eqaution: cannot be decomposed
 y = exp(x) + x^2

No compound expressions A-normal form


Seperator - a piar of contractors one for S, for for it's completment/





In [ ]:
def sivia(f, start, S, eps = 0.1):
    queue = [start]
    Xminux = []
    Xplus = []
    while length(queue) > 0:
        box = queue.pop()
        outer = f(box)
        if outer in outset:
            Xminus.append(outer)
        elif intersect(outer, S) == 0:
            pass
        elif width(outset) <= eps:
            Xplus.append(outset)
        else:
            queue.append(box.split())
    return Xminus, Xplus

# Wait so this does have all three

def contract(x,y,z): # x + y == z
    z = z.intersect( x + y )
    y = y.intersect( z - x )
    x = x.intersect( z - y )
    